In [31]:
import json
import pandas as pd
from bs4 import BeautifulSoup


In [32]:
train_path = 'SCOTUS/train.json'
dev_path = 'SCOTUS/dev.json'

In [33]:
with open(train_path, 'r', encoding="utf-8") as f:
    train = json.load(f)

html_content = train[0]["raw_source"]

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_content, 'html.parser')

# On suppose que le contenu intéressant ne se trouve que dans cette classe
div_class = "-display-inline-block text-left"
target_div = soup.find('div', class_=div_class)

# Extraire le texte du div en fusionnant les textes des balises <a> et <em> avec les textes précédent et suivant
if target_div:
    temp_result = []
    to_merge = False
    for element in target_div.children:
        if element.name == "a" or element.name == "em" :
            temp_result.append(element.get_text())
            to_merge = True
        elif (to_merge) :
            temp_result[-2] = temp_result[-2] + " " + temp_result[-1] + " " + element.get_text()
            temp_result.pop()
            to_merge = False
        else :
            temp_result.append(element.get_text())

else:
    print(f"Aucun div trouvé avec la classe '{div_class}'.")

result = []
for res in temp_result:
    if '\t' in res:
        string = res.replace('\t', '').replace('\n', '')
        result.append(string)
    else:
        result[-1] = result[-1] + " " + res.replace('\n', '')
    
for res in result:
    print(res)
 
